# ( A1.4 Selección de características)         
                                                                                                       
                                                                            Fernando Ariel Lozano Castillo
1. Primero importaremos los datos del archivo **Vino Tinto.csv**  en la variable `df` y  revisaremos las dimensiones del data frame e imprimiremos en consola tanto dichas dimensiones  con la función `shape`como las primeras 5 filas
de datos con la función `head`.                                                          

In [541]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


# Suprimir la advertencia
warnings.filterwarnings("ignore", category=FutureWarning)

# Leemos el archivo
df = pd.read_csv("Vino Tinto.csv")
print("Dimensiones :", df.shape , "\n")

# Imprimimos las primeras 5 filas
print(df.head(5))

Dimensiones : (1599, 12) 

   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        5  
3      

2. Ahora separaremos el data frame en **datos de entrenamiento** dentro de la variable `train` y **datos de prueba** en la variable `test` con una proporción de *80/20*. Es decir que ***80% para entrenar*** el modelo y el ***resto para validar los resultados*** con una partición aleatoria con el operador `sample` de pandas y el operador `drop` e `index`  para generar a `test` sin los elementos usados para el subset de entrenamiento.

In [499]:


train, test = train_test_split(df, train_size = 0.8)

#Imprime las dimensiones de los datos de entrenamiento
print("Dimensiones de Train: ",train.shape)

#Imprime las dimensiones de los datos de prueba
print("Dimensiones de Test: ", test.shape)

print(" ")

Dimensiones de Train:  (1279, 12)
Dimensiones de Test:  (320, 12)
 


Posteriormente, imprimiremos la cantidad de observaciones de cada conjunto de datos y comprobamos  si la suma de elementos en train y test es igual a la del dataset original.

In [502]:
if (train.shape[0] + test.shape[0] == df.shape[0]):
 print(" La división de los datos es correcta.")
else:
 print(" Hay una discrepancia en la división de los datos.","/n")

 La división de los datos es correcta.


3. **Generamos** la metodología de **selección hacia adelante** e imprime en consola los índices o  los nombres de las características seleccionadas. Para realizar este proceso, 
usaremos la función `“SequentialFeatureSelector”` de la librería 
`“mlxtend.feature_selection”`.
Aquí dejo una descripción de los 
parámetros que se usaron:<br>
- **a. estimator**: Un modelo de regresión lineal. Te recomiendo usar la función `LinearRegression` de la librería `sklearn.linear_model`.  
- **b. k_features**: Se puede seleccionar la cantidad de variables de salida que se desean, pero te recomiendo mejor usar un rango, y que el algoritmo determine el número adecuado. Por ejemplo, puedes definir el parámetro como `(2, 8)`, si te interesa que el método seleccione entre 2 y 8 variables.  
- **c. forward**: Determina si se hace selección hacia adelante (`True`) o hacia atrás (`False`); en este caso queremos hacer selección hacia adelante.  
- **d. scoring**: La métrica que se usará para determinar si un modelo es mejor que otro. Te recomiendo definirla como `'r2'` para usar la R cuadrada.  
- **e. cv**: Si se desea realizar validación cruzada, y cuántas instancias de la misma. Te recomiendo definir este parámetro como `10`.  

In [539]:

#Separa los datos en x y y de la serie de datos de entrenamiento
x_train = train.drop("calidad", axis= 1)
y_train = train["calidad"]


#modelo de regresion lineal
model = LinearRegression()

#caracteristicas del modelo de sequential future selector
sfs = SFS(model, k_features= (2,8), forward=True, scoring= 'r2', cv = 10 )

sfs.fit(x_train,y_train)


features = list(sfs.k_feature_names_)
print("Variables selecionadas: \n", features)

Variables selecionadas: 
 ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


4. Entrenamos un modelo que solamente contenga las variables seleccionadas, predecir la 
respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo 
usando la **R cuadrada**, posteriormente imprimiendo dicho valor en consola.<br>
- Para el primer paso, 
simplemente, usaremos la función `fit `en el modelo de **regresión lineal creado 
previamente**, asegurándonos de no introducir toda la información de X, sino solo de las 
variables seleccionadas.<br>
- Para **realizar las predicciones**,  usaremos la función `predict` en 
los datos de prueba, para dichos datos también seleccionaremos solo las 
variables de interés.<br>
- Para el último paso, usaremos la función `r2_score de 
sklearn.metrics.` 

In [531]:

y_test = test.calidad
x_test = test.drop("calidad", axis = 1)

x_trainF = x_train[features]
x_testF = x_test[features]

model.fit(x_trainF , y_train)

ypredict = model.predict(x_testF)

r2_forward = r2_score(y_test , ypredict)
print("R^2 =", r2_forward)

R^2 = 0.3774037891991848


5. Realizaremos ahora un proceso de **selección hacia atrás** a partir de las variables seleccionadas por el método de selección hacia adelante e imprimiremos en consola los índices o nombres de las 
variables seleccionadas.<br> Para realizar este proceso, usaremos la misma función 
del paso 3, pero definiendo ahora `forward=False.` También especificaremos una 
menor cantidad de variables posibles, por ejemplo: `k_features=(2,5). `

In [535]:

model1 = LinearRegression()

sfs1 = SFS(model, k_features= (2,5), forward=False, scoring= 'r2', cv = 10 )
#Ejecutamos el selector 
sfs1.fit(x_trainF,y_train)


features1 = list(sfs1.k_feature_names_)
print("Variables selecionadas: \n", features1)


Variables selecionadas: 
 ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


6. Repetimos el paso 4, pero para un modelo que contenga solamente las variables 
seleccionadas en el paso 5.<br> Imprimo en pantalla un breve texto que describa mi opinión 
sobre la diferencia en R cuadrada medida entre los modelos de los pasos 4 y 6 respondiendo a las preguntas,<br> **¿cuál 
modelo consideras que es mejor? ¿Por qué?** 

In [537]:
x_trainB = x_train[features1]
x_testB = x_test[features1]

model1.fit(x_trainB,y_train)

ypredict1 = model1.predict(x_testB)

r2_backward = r2_score(y_test,ypredict1)
print("R^2 de la selección hacia atrás =", r2_backward)



# Comparación entre ambos modelos
print("\nComparación de R cuadrada entre los modelos:", " \n")
print(f"Modelo hacia adelante: {r2_forward}")
print(" ")
print(f"Modelo hacia atrás: {r2_backward}")
print(" ")

if r2_forward > r2_backward:
    print("El modelo con selección hacia adelante tiene mejor capacidad predictiva.", " \n")
elif r2_forward < r2_backward:
    print("El modelo con selección hacia atrás tiene mejor capacidad predictiva.\n")
else:
    print("Ambos modelos tienen la misma capacidad predictiva.\n")

R^2 de la selección hacia atrás = 0.36673629131103824

Comparación de R cuadrada entre los modelos:  

Modelo hacia adelante: 0.3774037891991848
 
Modelo hacia atrás: 0.36673629131103824
 
El modelo con selección hacia adelante tiene mejor capacidad predictiva.  



Considero que es mejor el modelo hacia adelante debido a que su R^2 es mayor al del modelo hacia atrás. Al quitar variables, el R^2 no aumentó, lo que indica que las variables eliminadas no eran significativas para la predicción. En caso de que el modelo hacia atrás, con menos variables, hubiera tenido un mejor R^2, entonces habría sido la mejor opción porque se logró reducir a un modelo que contiene las variables con mayor correlación.

**Firma de honor** :  Doy mi palabra que he realizado esta actividad con integridad académica